А теперь давайте для сравнения сделаем то же самое, но в Keras. Суть процесса, конечно, не меняется, но кода получается заметно меньше. Как и в TensorFlow, в Keras набор данных MNIST можно загрузить средствами самой библиотеки:

In [10]:
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras.utils import np_utils

(X_train, y_train), (X_test, y_test) = mnist.load_data()

Затем давайте подготовим данные к тому, чтобы подать их на вход сети. Здесь нам потребуются три вспомогательные процедуры. Во-первых, нужно будет, как и раньше, превратить каждую картинку в двумерный массив:

In [11]:
batch_size, img_rows, img_cols = 64, 28, 28
X_train = X_train.reshape(X_train.shape[0], img_rows, img_cols, 1)
X_test = X_test.reshape(X_test.shape[0], img_rows, img_cols, 1)
input_shape = (img_rows, img_cols, 1)

Во-вторых, входные данные MNIST в Keras, в отличие от TensorFlow, представляют собой целые числа от 0 до 255; можно было бы обучать сеть и на таких данных, но давайте для единообразия все-таки приведем их к типу flo a t3 2 и нормализуем от 0 до 1, как раньше:

In [12]:
X_train = X_train.astype("float32")
X_test = X_test.astype("float32")
X_train /= 255
X_test /= 255

В-третьих, переведем правильные ответы в one-hot представление; для этого служит вспомогательная процедура to_categorical из keras.np_utils:

In [13]:
Y_train = np_utils.to_categorical(y_train, 10)
Y_test = np_utils.to_categorical(y_test, 10)

Теперь пора задавать собственно модель. В Keras это выглядит примерно так же, как в TensorFlow, но некоторые названия и параметры более узнаваемы. Сначала инициализируем модель:

In [14]:
model = Sequential()

Теперь добавим сверточные слои. Нам понадобится слой Convolution2D, основными аргументами которого являются число фильтров и размер окна, аргумент border_mode имеет ровно тот же смысл, что аргумент padding в TensorFlow (а теперь и в Keras это padding), а аргумент input_shape сообщает Keras, какой размерности тензор ожидать на входе:

In [18]:
model.add(Conv2D(filters=32, kernel_size=(5, 5), strides=(5, 5), 
                 padding="same", input_shape=input_shape))
model.add(Activation("relu"))
model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2), padding="same"))
model.add(Conv2D(filters=64, kernel_size=(5, 5), strides=(5, 5), 
                 padding="same", input_shape=input_shape))
model.add(Activation("relu"))
model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2), padding="same"))

В этом примере мы сохранили ту же архитектуру, что была выше. Обратите внимание, что в Keras нам не нужно отдельно инициализировать переменные, которые будут затем использоваться в качестве весов или свободных членов в слоях: Keras сам понимает, сколько должно быть весов у той или иной сверточной архитектуры, и сам поймет, что по ним нужно модель оптимизировать. Единственное, чем ему нужно для этого помочь, — задать явно размерность input_shape.  

Полносвязные слои тоже получаются достаточно просто. За «переформатирование» тензора, которое в TensorFlow делалось функцией tf.reshape, теперь отвечает дополнительный «слой», который называется Flatten и способен сам понять, тензор какой формы подается ему на вход:

In [19]:
model.add(Flatten())
model.add(Dense(1024))
model.add(Activation("relu"))
model.add(Dropout(0.5))
model.add(Dense(10))
model.add(Activation("softmax"))

И все, можно компилировать и запускать обучение:

In [21]:
model.compile(loss="categorical_crossentropy", optimizer="adam", metrics=["accuracy"])
model.fit(X_train, Y_train, batch_size=batch_size, epochs=10, verbose=1, 
          validation_data=(X_test, Y_test))
score = model.evaluate(X_test, Y_test, verbose=0)
print("Test score: %f" % score[0])
print("Test accuracy: %f" % score[1])

Train on 60000 samples, validate on 10000 samples
Epoch 1/10
60000/60000 [==============================] - 33s 545us/step - loss: 0.0810 - acc: 0.9731 - val_loss: 0.1222 - val_acc: 0.9622
Epoch 2/10
60000/60000 [==============================] - 31s 516us/step - loss: 0.0733 - acc: 0.9752 - val_loss: 0.1153 - val_acc: 0.9669
Epoch 3/10
60000/60000 [==============================] - 32s 531us/step - loss: 0.0702 - acc: 0.9765 - val_loss: 0.1146 - val_acc: 0.9659
Epoch 4/10
60000/60000 [==============================] - 33s 549us/step - loss: 0.0631 - acc: 0.9784 - val_loss: 0.1190 - val_acc: 0.9677
Epoch 5/10
60000/60000 [==============================] - 32s 537us/step - loss: 0.0610 - acc: 0.9789 - val_loss: 0.1237 - val_acc: 0.9650
Epoch 6/10
60000/60000 [==============================] - 34s 564us/step - loss: 0.0557 - acc: 0.9813 - val_loss: 0.1262 - val_acc: 0.9648 ac
Epoch 7/10
60000/60000 [==============================] - 43s 719us/step - loss: 0.0554 - acc: 0.9813 - val_loss:

Чтобы избежать оверфиттинга, можно, конечно, написать цикл по эпохам обучения, проверять в этом цикле
ошибку на валидационном множестве, сохранять веса модели в файл, а затем выбрать файл, соответствующий самой лучшей эпохе. Но оказывается, что в Keras все это можно выразить буквально в аргументах функции fit! Вот как обычно выглядит запуск обучения сложных моделей на практике:   

Здесь аргумент callbacks позволяет задать функции, запускаемые после каждой эпохи обучения. Можно написать эти функции самому, а можно воспользоваться стандартными. Так, функция ModelCheckpoint из модуля keras.callbacks — это вспомогательная процедура, которая после каждой эпохи обучения сохраняет модель в файл, имя которого подается на вход, в данном случае model.hdf5. А параметр save_best_only=True позволяет после каждой эпохи проверять метрику качества, заданную в параметре monitor (в данном случае val_acc, то есть точность на валидационном множестве), и перезаписывать сохраняемую модель только в том случае, если эта метрика улучшилась. Обратите также внимание на параметр valldation_split=0.1: если у вас нет заранее выделенного тренировочного и тестового подмножеств (у нас они были сразу в датасете MNIST), то можно просто попросить Keras использовать для валидации случайное подмножество входных данных, составляющее заданную их долю, в данном случае 10 %.

In [22]:
from keras.callbacks import ModelCheckpoint
model.fit(X_train, Y_train, 
          callbacks=[ModelCheckpoint("model.hdf5", monitor="val_acc", save_best_only=True,
          save_weights_only=False, mode="auto")], batch_size=batch_size, epochs=10, verbose=1, 
          validation_data=(X_test, Y_test))
score = model.evaluate(X_test, Y_test, verbose=0)
print("Test score: %f" % score[0])
print("Test accuracy: %f" % score[1])

Train on 60000 samples, validate on 10000 samples
Epoch 1/10
60000/60000 [==============================] - 34s 566us/step - loss: 0.0441 - acc: 0.9845 - val_loss: 0.1298 - val_acc: 0.9664
Epoch 2/10
60000/60000 [==============================] - 34s 565us/step - loss: 0.0428 - acc: 0.9856 - val_loss: 0.1341 - val_acc: 0.9679
Epoch 3/10
60000/60000 [==============================] - 38s 626us/step - loss: 0.0404 - acc: 0.9862 - val_loss: 0.1440 - val_acc: 0.9669
Epoch 4/10
60000/60000 [==============================] - 32s 538us/step - loss: 0.0379 - acc: 0.9862 - val_loss: 0.1422 - val_acc: 0.9674
Epoch 5/10
60000/60000 [==============================] - 36s 598us/step - loss: 0.0352 - acc: 0.9879 - val_loss: 0.1485 - val_acc: 0.9669
Epoch 6/10
60000/60000 [==============================] - 34s 568us/step - loss: 0.0350 - acc: 0.9879 - val_loss: 0.1468 - val_acc: 0.9679
Epoch 7/10
60000/60000 [==============================] - 36s 603us/step - loss: 0.0363 - acc: 0.9875 - val_loss: 0.